In [6]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#data dimension will be: #bag * #frame * 244 * 244 * 3
#simple implementation of CNN>LSTM without multires


In [3]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/billy/Desktop/VideoClasBinned'
checkAndGeneratePath(parentPath)
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'



Length Check on Folder Succeed


In [3]:
trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

In [24]:
combo=prepdata(testPath)
len(combo)

124

In [4]:
#Using TimeDistributed Wrapper
inputData=Input(shape=(None, 244, 244, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 244, 244, 3))
for layer in model.layers[:278]: #until mixed9_0, untrainable
   layer.trainable = False
encoded_frame = TimeDistributed(model)(inputData)

encoded_frame=TimeDistributed(Flatten())(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = Dense(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)

intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(29, activation='softmax')(intermediateOut)
optmr = keras.optimizers.adam(lr=0.005, decay = 1e-5)
model_final = Model(input = [inputData], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 20,
                    workers=2)
    

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
399/399 [==============================] - 430s 1s/step - loss: 3.0937 - acc: 0.1153 - val_loss: 15.6939 - val_acc: 0.0263
Epoch 2/20
399/399 [==============================] - 270s 678ms/step - loss: 2.9869 - acc: 0.1303 - val_loss: 15.6939 - val_acc: 0.0263
Epoch 3/20
399/399 [==============================] - 267s 669ms/step - loss: 2.9976 - acc: 0.1253 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/20
399/399 [==============================] - 265s 665ms/step - loss: 2.9719 - acc: 0.1353 - val_loss: 15.6939 - val_acc: 0.0263
Epoch 5/20
399/399 [==============================] - 266s 666ms/step - loss: 2.9784 - acc: 0.1153 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/20
399/399 [==============================] - 266s 667ms/step - loss: 2.9791 - acc: 0.1253 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/20
399/399 [==============================] - 267s 669ms/step - loss: 2.9839 - acc: 0.1353 - val_loss: 16.1181 - 

In [9]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10912985240160912434
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7040294251406950459
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1823080448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13790727769911004398
physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14818599878350816964
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
model_json = model_final.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("model.h5")

In [7]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("model.h5")
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [11]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
with tf.device('GPU:0'):
    for i in range(124):
        x,y=testGen.__getitem__(i)
        pred=model_final.predict(x)
        if(pred==y):
            correct+=1
print('Accuracy of Testing Set: ',correct/124)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node time_distributed_1/conv2d_1/convolution}}]]
	 [[{{node dense_3/Softmax}}]]